In [1]:
# Genero un archivo de consulta para la función PlayTimeGenre que usaré en FastAPI

import pandas as pd
from ast import literal_eval

# Cargar el archivo CSV
user_items = pd.read_csv('../PI_ML_OPS_data util/user_items.csv', converters={'items': literal_eval})

# Inicializar un diccionario para almacenar la sumatoria de horas por item_id
sumatoria_horas_por_item = {}

# Iterar sobre las entradas del archivo user_items
for _, entrada in user_items.iterrows():
    # Iterar sobre los diccionarios en la columna 'items'
    for item in entrada['items']:
        # Obtener el item_id y las horas jugadas
        item_id = item.get('item_id')
        horas_jugadas = item.get('playtime_forever', 0)

        # Agregar las horas jugadas al diccionario por item_id
        if item_id in sumatoria_horas_por_item:
            sumatoria_horas_por_item[item_id] += horas_jugadas
        else:
            sumatoria_horas_por_item[item_id] = horas_jugadas

# Crear un DataFrame a partir del diccionario
sumatoria_horas_df = pd.DataFrame(list(sumatoria_horas_por_item.items()), columns=['item_id', 'sumatoria_horas'])

# Guardar el nuevo archivo CSV
sumatoria_horas_df.to_csv('sumatoria_horas_por_item.csv', index=False)

KeyboardInterrupt: 

In [53]:
import pandas as pd

# Cargar el archivo CSV
data_games_ = pd.read_csv('../PI_ML_OPS_data util/steam_games.csv')

# Filtrar registros completos con prioridad en 'release_date'
data_games_cleaned = data_games_.dropna(subset=['genres', 'release_date', 'id'])

# Seleccionar solo las columnas 'genres', 'release_date' y 'id'
data_games_cleaned = data_games_cleaned[['genres', 'release_date', 'id']]

# Mostrar información del DataFrame resultante
data_games_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 28833 entries, 0 to 32133
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28833 non-null  object 
 1   release_date  28833 non-null  object 
 2   id            28833 non-null  float64
dtypes: float64(1), object(2)
memory usage: 901.0+ KB


In [55]:
# Convertir 'release_date' a formato de fecha y obtener el año
data_games_cleaned['release_date'] = pd.to_datetime(data_games_cleaned['release_date'], errors='coerce')
data_games_cleaned['year'] = data_games_cleaned['release_date'].dt.year

# Eliminar la columna 'release_date' original
data_games_cleaned.drop(columns=['release_date'], inplace=True)

# Mostrar información del DataFrame resultante
data_games_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Index: 28833 entries, 0 to 32133
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   genres  28833 non-null  object 
 1   id      28833 non-null  float64
 2   year    28548 non-null  float64
dtypes: float64(2), object(1)
memory usage: 901.0+ KB


In [ ]:
# Eliminar los registros que no contengan valores en la columna year
data_games_cleaned = data_games_cleaned.dropna(subset=['year'])

In [63]:
# Cargar el archivo 'sumatoria_horas_por_item.csv' y fusionar DataFrames:

sumatoria_horas = pd.read_csv('sumatoria_horas_por_item.csv')
result = pd.merge(data_games_cleaned[['id', 'genres', 'year']], sumatoria_horas[['item_id', 'sumatoria_horas']], left_on='id', right_on='item_id', how='inner')


In [65]:
result

,id,genres,year,item_id,sumatoria_horas
0,282010.0,"['Action', 'Indie', 'Racing']",1997.0,282010,9319
1,70.0,['Action'],1998.0,70,2682852
2,1640.0,['Strategy'],2006.0,1640,27397
3,1630.0,['Strategy'],2006.0,1630,21111
4,2400.0,"['Action', 'Indie', 'RPG']",2006.0,2400,1048156
...,...,...,...,...,...
8622,259340.0,"['Action', 'Casual', 'Indie']",2003.0,259340,371
8623,30.0,['Action'],2003.0,30,758991
8624,200980.0,"['Strategy', 'RPG', 'Indie']",2003.0,200980,3393
8625,13230.0,['Action'],2004.0,13230,349119


In [66]:
# Dejo sólo las columnas que me importan
result = result.drop(columns=['id', 'item_id'])


In [69]:
# Genero un explode sobre la columna genres para obtener una fila por genero
import ast

result['genres'] = result['genres'].apply(ast.literal_eval)
result_exploded = result.explode('genres')


In [70]:
result_exploded

,genres,year,sumatoria_horas
0,Action,1997.0,9319
0,Indie,1997.0,9319
0,Racing,1997.0,9319
1,Action,1998.0,2682852
2,Strategy,2006.0,27397
...,...,...,...
8624,Strategy,2003.0,3393
8624,RPG,2003.0,3393
8624,Indie,2003.0,3393
8625,Action,2004.0,349119


In [71]:
# Agrupo por genero
result_grouped = result_exploded.groupby(['genres', 'year'])['sumatoria_horas'].sum().reset_index()


In [73]:
result_grouped

,genres,year,sumatoria_horas
0,Action,1983.0,3582
1,Action,1984.0,384
2,Action,1988.0,16243
3,Action,1989.0,607
4,Action,1990.0,18787
...,...,...,...
338,Web Publishing,2013.0,335849
339,Web Publishing,2014.0,33732
340,Web Publishing,2015.0,348861
341,Web Publishing,2016.0,136


In [72]:
# Creo archivo para ser utilizado por la función dentro de FastAPI
result_grouped.to_csv('PlayTimeGenre.csv', index=False)
